#  1절. SQLite 데이터베이스 연결

## 1.1 SQLite와 파이썬
- SQLite 데이터페이스는 별도의 서버 프로세스 없이 SQL을 사용하여 DB엑세스할 수 있도록 만든 간단한 **디스크 기반 데이터베이스를 제공** 하는 C 라이브러리

- SQLite 는 프로토타입(시제품 나오기 전 제품)을 만들때까지 사용. 정식제품은 오라클이나 다른 데이터베이스를 이식

- db browser for sqlite https://sqlitebrowser.org/dl/

In [1]:
import sqlite3
sqlite3.sqlite_version

'3.33.0'

## 1.2 데이터베이스 연결 

In [11]:
# DB 연결 객체 생성시, 파일이 없으면 파일 생성, 파일이 있으면 그 파일 연결
conn = sqlite3.connect('data/ch10_example.db')

In [7]:
conn.close()

In [12]:
#커서 객ㅊ 반환. 커서는 SQL문을 실행시키고 결과 데이터를 조회하는데 사용
cursor = conn.cursor()

In [13]:
cursor.execute("""
        CREATE TABLE MEMBER(
            NAME TEXT,
            AGE INT,
            EMAIL TEXT
        )
""")

In [14]:
cursor.execute("INSERT INTO MEMBER VALUES ('홍길동',20,'H@hong.com')")
cursor.execute("INSERT INTO MEMBER VALUES ('최민재',23,'H@hong.com')")
cursor.execute("INSERT INTO MEMBER VALUES ('윤현우',30,'H@hong.com')")

In [16]:
conn.commit() #sqlite은 commit이 별 의미가 없다.

In [17]:
cursor.execute('SELECT * FROM MEMBER') #실행 결고는 cursor가 가리킴

In [18]:
#select문의 결과를 받는 함수 : 
##fetchall() -결과를 모두 받을 때
##fetchone() -결과를 한 행씩 받을때
##fetchmay(n) - 결과를 n행 받을 때
print(cursor.fetchall())

[('홍길동', 20, 'H@hong.com'), ('최민재', 23, 'H@hong.com'), ('윤현우', 30, 'H@hong.com')]


In [19]:
print(cursor.fetchall())

[]


In [22]:
cursor.execute("SELECT * FROM MEMBER")
members = cursor.fetchall()
for member in members:
    print(member)

('홍길동', 20, 'H@hong.com')
('최민재', 23, 'H@hong.com')
('윤현우', 30, 'H@hong.com')


In [23]:
members[0]

('홍길동', 20, 'H@hong.com')

In [27]:
cursor.execute("SELECT * FROM MEMBER")
print(cursor.fetchone())
print(cursor.fetchone())
print(cursor.fetchone())
print(cursor.fetchone())

('홍길동', 20, 'H@hong.com')
('최민재', 23, 'H@hong.com')
('윤현우', 30, 'H@hong.com')
None


In [28]:
cursor.execute("SELECT * FROM MEMBER")
while True:
    member=cursor.fetchone()
    if member is None:
        break;
    print(member)

('홍길동', 20, 'H@hong.com')
('최민재', 23, 'H@hong.com')
('윤현우', 30, 'H@hong.com')


In [34]:
cursor.execute("SELECT * FROM MEMBER")
for member in cursor.fetchmany(2):
    print(member)
    print(member[0],member[1],member[2])

('홍길동', 20, 'H@hong.com')
홍길동 20 H@hong.com
('최민재', 23, 'H@hong.com')
최민재 23 H@hong.com


In [35]:
cursor.fetchmany(2) #위에서 2개만 뿌리고 커서가 멈춰있는 곳에서 내보냄.

[('윤현우', 30, 'H@hong.com')]

In [36]:
cursor.close() #생략 가능
conn.close()

## 1.3 SQL 구문에 따라 파라미터 사용하기
- qmark, named 방식(네임드 추천)

In [41]:
#파라미터 사용하기 : qmark방법 이용
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)",('최민재','윤현우'))
print(cursor.fetchall())

[('최민재', 23, 'H@hong.com'), ('윤현우', 30, 'H@hong.com')]


In [45]:
#튜플로
name1=input('검색할 이름1:')
name2=input('검색할 이름2:')
names=(name1,name2)
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)",names)
print(cursor.fetchall())

검색할 이름1:최민재
검색할 이름2:윤현우
[('최민재', 23, 'H@hong.com'), ('윤현우', 30, 'H@hong.com')]


In [53]:
#파라미터 사용하기: named 방법
name=input('검색하고자 하는 이름:')
cursor.execute("SELECT * FROM MEMBER WHERE NAME=:name",{'name':name})
members = cursor.fetchall()
if len(members)==0 :
      print('해당이름데이터가없습니다.')
else:
    print(cursor.fetchall())

검색하고자 하는 이름:김정길
해당이름데이터가없습니다.


In [55]:
try:
    name = input('입력할 이름?')
    age = int(input('나이?'))
    email = input('입력할 메일?')
except:
    print('나이를 잘못 입력했어요. 초기값 20살')
    age=20
    email=input('입력 메일')
cursor.execute("INSERT INTO MEMBER VALUES (:name,:age,:email)",
              {'name':name,'age':age,'email':email})

입력할 이름?김
나이?20
입력할 메일?ㅋㅋㅋ


In [58]:
cursor.execute('select * from member')
print(cursor.fetchall())

[('홍길동', 20, 'H@hong.com'), ('최민재', 23, 'H@hong.com'), ('윤현우', 30, 'H@hong.com'), ('김', 20, 'ㅋㅋㅋ')]


In [60]:
cursor.close()
conn.close()

# 2절. 오라클 데이터베이스 연결.

In [63]:
import cx_Oracle
oracle_dsn=cx_Oracle.makedsn(host="localhost",port=1521,sid='xe')
oracle_dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))'

In [65]:
conn = cx_Oracle.connect("scott","tiger",dsn=oracle_dsn)
conn
#여기서 에러날경우 VC_redist 인스톨할것

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [66]:
#cursor 얻어오기
cursor=conn.cursor()

In [67]:
sql="SELECT * FROM EMP"
cursor.execute(sql)
emp=cursor.fetchall()

In [68]:
for e in emp:
    print(e)

(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7499, 'ALLEN', 'SALESMAN', 7698, datetime.datetime(1981, 2, 20, 0, 0), 1600.0, 300.0, 30)
(7521, 'WARD', 'SALESMAN', 7698, datetime.datetime(1981, 2, 22, 0, 0), 1250.0, 500.0, 30)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7654, 'MARTIN', 'SALESMAN', 7698, datetime.datetime(1981, 9, 28, 0, 0), 1250.0, 1400.0, 30)
(7698, 'BLAKE', 'MANAGER', 7839, datetime.datetime(1981, 5, 1, 0, 0), 2850.0, None, 30)
(7782, 'CLARK', 'MANAGER', 7839, datetime.datetime(1981, 6, 9, 0, 0), 2450.0, None, 10)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7839, 'KING', 'PRESIDENT', None, datetime.datetime(1981, 11, 17, 0, 0), 5000.0, None, 10)
(7844, 'TURNER', 'SALESMAN', 7698, datetime.datetime(1981, 9, 8, 0, 0), 1500.0, 0.0, 30)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7900, 'JAMES', 'CL

In [73]:
sql="SELECT * FROM EMP WHERE DEPTNO=:deptno"
deptno=input('검색하려는 부서번호?')
cursor.execute(sql,{'deptno':deptno})

emp=cursor.fetchall()
if len(emp)==0:
    print('데이터없음')
else:
    for e in emp:
        print(e)

검색하려는 부서번호?20
(7369, 'SMITH', 'CLERK', 7902, datetime.datetime(1980, 12, 17, 0, 0), 800.0, None, 20)
(7566, 'JONES', 'MANAGER', 7839, datetime.datetime(1981, 4, 2, 0, 0), 2975.0, None, 20)
(7788, 'SCOTT', 'ANALYST', 7566, datetime.datetime(1982, 12, 9, 0, 0), 3000.0, None, 20)
(7876, 'ADAMS', 'CLERK', 7788, datetime.datetime(1983, 1, 12, 0, 0), 1100.0, None, 20)
(7902, 'FORD', 'ANALYST', 7566, datetime.datetime(1981, 12, 3, 0, 0), 3000.0, None, 20)


In [74]:
import pandas as pd
cursor.execute("SELECT * FROM EMP")
data=cursor.fetchall() #튜플 리스트
data_df=pd.DataFrame(data) #데이터 프레임 타입이 됨
data_df

,0,1,2,3,4,5,6,7
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30
5,7698,BLAKE,MANAGER,7839.0,1981-05-01,2850.0,NaN,30
6,7782,CLARK,MANAGER,7839.0,1981-06-09,2450.0,NaN,10
7,7788,SCOTT,ANALYST,7566.0,1982-12-09,3000.0,NaN,20
8,7839,KING,PRESIDENT,NaN,1981-11-17,5000.0,NaN,10
9,7844,TURNER,SALESMAN,7698.0,1981-09-08,1500.0,0.0,30


In [76]:
#각 필드 특징 정보(필드명, type, display_size, 내부크기, 정확도, scale, null포함하는지아닌지)
cursor.description
[row[0]for row in cursor.description]

['EMPNO', 'ENAME', 'JOB', 'MGR', 'HIREDATE', 'SAL', 'COMM', 'DEPTNO']

In [79]:
data_df.columns=[row[0]for row in cursor.description]

In [82]:
data_df.head()

,EMPNO,ENAME,JOB,MGR,HIREDATE,SAL,COMM,DEPTNO
0,7369,SMITH,CLERK,7902.0,1980-12-17,800.0,NaN,20
1,7499,ALLEN,SALESMAN,7698.0,1981-02-20,1600.0,300.0,30
2,7521,WARD,SALESMAN,7698.0,1981-02-22,1250.0,500.0,30
3,7566,JONES,MANAGER,7839.0,1981-04-02,2975.0,NaN,20
4,7654,MARTIN,SALESMAN,7698.0,1981-09-28,1250.0,1400.0,30


In [83]:
cursor.close()
conn.close()

# 3절. MariaDB 데이터베이스 연결

In [91]:
!pip install pymysql
import pymysql

In [92]:
conn =pymysql.connect(host="localhost",
                      port=3306,
                      db='kimdb',user='root',passwd='mysql',
                      charset='utf8',autocommit=True)
conn

In [93]:
cursor = conn.cursor()
sql="select * from personal"
cursor.execute(sql)
result = cursor.fetchall()
for r in result:
    print(r)

(1001, 'bill', 'president', None, datetime.date(1989, 1, 10), 7000, None, 10)
(1111, 'smith', 'manager', 1001, datetime.date(1990, 12, 17), 1000, None, 10)
(1112, 'ally', 'salesman', 1116, datetime.date(1991, 2, 20), 1600, 500, 30)
(1113, 'word', 'salesman', 1116, datetime.date(1992, 2, 24), 1450, 300, 30)
(1114, 'james', 'manager', 1001, datetime.date(1990, 4, 12), 3975, None, 20)
(1116, 'johnson', 'manager', 1001, datetime.date(1991, 5, 1), 3550, None, 30)
(1118, 'martin', 'analyst', 1111, datetime.date(1991, 9, 9), 3450, None, 10)
(1121, 'kim', 'clerk', 1114, datetime.date(1990, 12, 8), 4000, None, 20)
(1123, 'lee', 'salesman', 1116, datetime.date(1991, 9, 23), 1200, 0, 30)
(1226, 'park', 'analyst', 1111, datetime.date(1990, 1, 3), 2500, None, 10)


In [94]:
personal = pd.DataFrame(result)
personal.colums=[row[0] for row in cursor.description]
personal.head()

<ipython-input-94-40f5623fe0bd>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  personal.colums=[row[0] for row in cursor.description]


,0,1,2,3,4,5,6,7
0,1001,bill,president,NaN,1989-01-10,7000,NaN,10
1,1111,smith,manager,1001.0,1990-12-17,1000,NaN,10
2,1112,ally,salesman,1116.0,1991-02-20,1600,500.0,30
3,1113,word,salesman,1116.0,1992-02-24,1450,300.0,30
4,1114,james,manager,1001.0,1990-04-12,3975,NaN,20


In [95]:
cursor.close()
conn.close()

In [ ]:
# 4절. 회원 입력/출력/삭제/검색 애플리케이션
-SQLite 이용, Oracle 이용